In [1]:
#Excerpt from COMS 4995 Assignment #1
#Part 2: CNN Using Pytorch
#Kaveri Thakoor, Linyong Nan
#Colab Notebook URL: 
#https://colab.research.google.com/drive/1Y89IVS00CxRuRQZ6PG4YqZlNfbafTNNY

#Code to import correct version of pytorch
!cat /etc/*-release

from google.colab import drive
drive.mount('/gdrive')

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=18.04
DISTRIB_CODENAME=bionic
DISTRIB_DESCRIPTION="Ubuntu 18.04.1 LTS"
NAME="Ubuntu"
VERSION="18.04.1 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.1 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········

In [1]:
import sys
print (sys.version)

from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available()) 

3.6.6 (default, Sep 12 2018, 18:26:19) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]
tcmalloc: large alloc 1073750016 bytes == 0x5c060000 @  0x7f5fecb752a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
0.4.0
True


In [8]:
#!easy_install-2.7 --user PIL
!pip install --no-cache-dir -I pillow
!pip3 install imageio

import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import glob
import imageio
from torch.utils import data
# path1 is the path to the folder of images with disease 
# path2 is the path to the folder of images without disease 
path1 = "/gdrive/My Drive/new_dataset/trainProbMaps/patients/*"
path2 = "/gdrive/My Drive/new_dataset/trainProb/controls/*"
path3 = "/gdrive/My Drive/new_dataset/valProbMaps/patients/*"
path4 = "/gdrive/My Drive/new_dataset/valProbMaps/controls/*"

class data_reader(data.Dataset):
    def __init__(self, mode):
        self.mode = mode
        self.train_list = []
        self.val_list = []
        if self.mode == 'train':
            files1 = glob.glob(path1)
            files2 = glob.glob(path2)
            for i in files1:
                self.train_list.append((i,0))
            for i in files2:
                self.train_list.append((i,1))
            print (len(self.train_list))
        elif self.mode == 'val':
            files1 = glob.glob(path3)
            files2 = glob.glob(path4)                  
            for i in files1:
                self.val_list.append((i,0))
            for i in files2:
                self.val_list.append((i,1)) 
            print (len(self.val_list))
        else:
            print('mode can only be train or val')
        

    def __getitem__(self,index):
        #print ('here')
         #self.mode = mode
        if self.mode == 'train':
            filepath = self.train_list[index][0]
            label = self.train_list[index][1]
            img = imageio.imread(filepath)    
            img = img.transpose(2,0,1)
        elif self.mode == 'val':
            filepath = self.val_list[index][0]
            label = self.val_list[index][1]
            img = imageio.imread(filepath) 
            img = img.transpose(2,0,1)
        #print (img.shape)
        return np.array(img), label
    
    def __len__(self):
        return len(self.train_list)
        
#     def augment(self,I):
#       # write code to augment image here - random crops, random flips etc
#       # if you dont want to augment images, leave this as it is
#       pass
#       return I

# to use the dataloader

batch_size = 25

train_reader = data_reader(mode='train')
#val_reader = data_reader(mode='val')
#epochs = 15

for epoch in range(15):
    train_dataloader = data.DataLoader(train_reader,
                batch_size=batch_size,
                shuffle=False,
                num_workers=1,
                drop_last=False)
    
# for epoch in range(15):
#     val_dataloader = data.DataLoader(val_reader,
#             batch_size=batch_size,
#             shuffle=False,
#             num_workers=1,
#             drop_last=False)


for images, labels in train_dataloader:
    print (images.size())
# for images, labels in val_dataloader:
#     print(images.size())

    100% |████████████████████████████████| 2.0MB 20.6MB/s 
195


AttributeError: ignored

In [0]:
#First Model Variation: ConvNetModelBest
class ConvNetModelBest(nn.Module):
    def __init__(self):
        super(ConvNetModelBest, self).__init__()
        # input is 32x32
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        # feature map size is 16*16 by pooling
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # feature map size is 8*8 by pooling
        #Adding third conv layer
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        # feature map size is 4*4 by pooling
        self.fc1 = nn.Linear(128*4*4, 1024)
        self.fc2 = nn.Linear(1024, 10)
        
    def forward(self, x):
        # x = (-1,1,28,28) --> (-1, 3, 32, 32) [MNIST dims --> CIFAR10 dims]
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) 
        # x = (-1,32,14,14) --> (-1, 64, 16, 16)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # x = (-1,64,7,7) --> (-1, 64, 8, 8)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        # x = (-1,64,7,7) --> (-1, 128, 4, 4)
        x = x.view(-1, 128*4*4) #reshape before sending to fully-connected layer
        # x = (-1,64*7*7) --> (-1, 128, 4, 4)
        x = F.relu(self.fc1(x))
        # x = (-1,1024) --> (-1, 2048)
        x = F.dropout(x, training=self.training)
        # x = (-1,1024) --> (-1, 2048)
        x = self.fc2(x)
        # x = (-1,10) --> (-1, 10)
        return F.log_softmax(x,1)
      
model = ConvNetModelBest()
#GPU: 
#model.cuda()

#ConvNetModelBest Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

# Training the model
# model.train()  # to set train mode for drop out
# train_loss = []
# train_accu = []
# i = 0
# for epoch in range(15):
#     count = 0
#     for data, target in train_loader:
#       #GPU: 
#       data, target = data.cuda(), target.cuda()
#       optimizer.zero_grad()
#       output = model(data) # calls the forward function of model, is equivalent to model.forward(data)
#       loss = F.nll_loss(output, target)
#       loss.backward()    # calc gradients
#       train_loss.append(loss.item())
#       optimizer.step()   # update gradients
#       prediction = output.data.max(1)[1]   # first column has actual prob.
#       accuracy = np.sum(prediction.cpu().numpy()==target.cpu().numpy())/batch_size*100
#       train_accu.append(accuracy)
#       if i % 1000 == 0:
#         print('Train Step: {}\tLoss: {:.3f}\tAccuracy: {:.3f}'.format(i, loss.item(), accuracy))
#       i += 1

# Part 2.4: Training Accuracy     
# model.eval()
# correctBest = 0
# count = 0
# for data, target in valid_loader:
#   with torch.no_grad(): # so that computation graph history is not stored
#     data, target = data.cuda(), target.cuda()
#     output = model(data)
#     prediction = output.data.max(1)[1]
#     correctBest += prediction.eq(target.data).sum()

In [0]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
#matplotlib inline
import numpy as np


import torchvision.models as models

alexnet = models.alexnet(pretrained=True)

class AlexNetTrial(nn.Module):
  
    def __init__(self):
        super(AlexNetTrial, self).__init__()
        #features = list(vgg16(pretrained = True).features)[:23]
        self.classifier = nn.Linear(512, 10) 

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return F.log_softmax(out,1)
        #x = self.features(x)
        #return x
            
#vgg16 = models.vgg16(pretrained=True)

model = alexnet
#GPU: 
#model.cuda()